In [183]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [6]:
#power provided by one hypha
rho_TAG = 0.18
eTAG = 30e-3 #uJ.um-3 30MJ.L-1
rho_e = rho_TAG*eTAG #uJ.um-3
r_hypha = 5 #um
v_transport = 7 #um.s-1
Ptrunk = np.pi*r_hypha**2*rho_e*v_transport #uW
Ptrunk

2.9688050576423546

In [11]:
r_vesicle = 1 #um
e_walking = 100 #ATP.um-1
e_ATP= 1.33e-13 #uJ
Pwalking =v_transport * e_walking*e_ATP #uW
p_transport = rho_TAG*Pwalking*r_hypha**2/(r_vesicle**3) #uW.um-1
L = 1e6 #um
P_transport = L*p_transport #uW
P_transport

0.0004189499999999999

In [18]:
L_prot = 400 #AA
n = 3e6 #um-3
Eproteinsynthesis = 4 #ATP.AA-1
e_total_prot_synth = Eproteinsynthesis*e_ATP*L_prot*n*np.pi*r_hypha**2 #uJ.um-1
v_colony_growth = 8 #um.s-1
P_total_protein_synthesis = e_total_prot_synth*v_colony_growth #uW
P_total_protein_synthesis

0.4011185500103447

In [249]:
class Energetic_model:
    def init(self,params):
        self.params = params
    def set_alpha(self,alpha):
        self.alpha = alpha
    def set_init_L(self,L):
        self.init_L = L
    def set_init_area(self,area):
        self.init_area = area
    def get_L(self,t):
        return(None)
    def get_L_exp(self,t):
        return(self.init_L*np.exp(self.alpha*t))
    def set_speed_growth(self,speed_growth):
        self.v = speed_growth
    def get_v_colony(self,t):
        y = self.get_L(t)
        dydx = np.gradient(y, t)
        return(dydx)
    def get_P_total_protein_synthesis(self,t):
        return(e_total_prot_synth*self.get_v_colony(t))
    def get_P_transport(self,t):
        return(p_transport*self.get_L(t))
    def get_area(self,t):
        return(np.pi/2*(self.v*t)**2+self.init_area)
    def get_area_ring(self,t):
        return(np.pi*(self.v*t))
    
class quadratic_model(Energetic_model):
    def set_density_sat(self,dens_sat):
        self.dens_sat = dens_sat
    def get_L(self,t):
        L_exp = self.get_L_exp(t)
        area = self.get_area(t)
        dens = L_exp/area
        is_above_sat = dens >=self.dens_sat
        return(area * self.dens_sat*is_above_sat+(1-is_above_sat)*self.get_L_exp(t))

class exp_model(Energetic_model):
    def get_L(self,t):
        return(self.get_L_exp(t))

In [280]:
np.min(data['tot_length_study']/data['area_sep_comp'])

963.1673183212598

In [300]:
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_plate_infos.pick"

time_plate_info = pd.read_pickle(path_save)
data = time_plate_info.loc[time_plate_info['Plate']==94]
quad_mod = quadratic_model()
alpha = 1.3/(24*3600)
v_growth = 300/3600
density_sat = 1250e-6
# density_sat = 6e-6
init_L = 20000
init_area = init_L/(np.max(data['tot_length_study']/data['area_sep_comp'])*1e-6)
quad_mod.set_alpha(alpha)
quad_mod.set_init_L(init_L)
quad_mod.set_init_area(init_area)

quad_mod.set_density_sat(density_sat)
quad_mod.set_speed_growth(v_growth)
exp_mod = exp_model()
exp_mod.set_alpha(alpha)
exp_mod.set_init_L(init_L)
exp_mod.set_speed_growth(v_growth)
exp_mod.set_init_area(init_area)



In [301]:
%matplotlib widget
num_days = 30
t = np.linspace(0,3600*24*num_days,100)
area = quad_mod.get_area(t)
fig, ax = plt.subplots()
ax.plot(area,quad_mod.get_L(t),label = 'quadratic')
ax.plot(area,exp_mod.get_L(t),label='exponential')
ax.scatter(data['area_sep_comp']*10**6,data['tot_length_study'],label='data',s=5)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('area($\mu m^{2}$)')
ax.set_ylabel('Length ($\mu m$)')
ax.vlines(np.max(data['area_sep_comp']*10**6),np.min(quad_mod.get_L(t)),10*np.max(quad_mod.get_L(t)),color="red")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [288]:
%matplotlib widget

fig, ax = plt.subplots()
ax.plot(area,quad_mod.get_L(t)/area,label = 'quadratic')
ax.plot(area,exp_mod.get_L(t)/area,label='exponential')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('area($\mu m^{2}$)')
ax.set_ylabel('density ($\mu m.\mu m^{-2}$)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [241]:
%matplotlib widget

fig, ax = plt.subplots()
y = quad_mod.get_P_transport(t)
ax.plot(area,quad_mod.get_P_transport(t),label = 'quadratic')
ax.plot(area,exp_mod.get_P_transport(t),label='exponential')
ax.plot(area,0*t+3,label = "Power provided by one trunk hypha")
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('area($\mu m^{2}$)')
ax.set_ylabel('Power required for transport ($\mu W$)')
ax.vlines(np.max(data['area_sep_comp']*10**6),np.min(y),10*np.max(y),color="red")

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [245]:
%matplotlib widget

fig, ax = plt.subplots()
y = quad_mod.get_P_total_protein_synthesis(t)
ax.plot(area,quad_mod.get_P_total_protein_synthesis(t),label = 'quadratic')
ax.plot(area,exp_mod.get_P_total_protein_synthesis(t),label='exponential')
ax.plot(area,t,label = "Power provided by one trunk hypha")

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('area($\mu m^{2}$)')
ax.set_ylabel('Power required for protein synthesis ($\mu W$)')
# ax.vlines(np.max(data['area_sep_comp']*10**6),np.min(y),10*np.max(y),color="red")

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …